In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 4.5 MB/s eta 0:00:00a 0:00:01


In [3]:
import zipfile

with zipfile.ZipFile('/kaggle/input/quora-question-pairs/train.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('/kaggle/working/')

In [4]:
with zipfile.ZipFile('/kaggle/input/quora-question-pairs/test.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('/kaggle/working/')

In [5]:
df = pd.read_csv('train.csv')
# test = pd.read_csv('test.csv',low_memory=False)

In [6]:
df.head(3)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0


In [7]:
df['is_duplicate'].value_counts()

is_duplicate
0    255027
1    149263
Name: count, dtype: int64

In [8]:
df[['question1','question2']].iloc[0].to_list()

['What is the step by step guide to invest in share market in india?',
 'What is the step by step guide to invest in share market?']

In [9]:
df=df[['question1','question2','is_duplicate']]

In [10]:
from sklearn.model_selection import train_test_split

train,test = train_test_split(df,test_size=0.05,random_state=1,stratify=df['is_duplicate'])
train,val = train_test_split(train,test_size=0.05,random_state=1,stratify=train['is_duplicate'])

In [11]:
print(train.shape)
print(val.shape)
print(test.shape)

(364871, 3)
(19204, 3)
(20215, 3)


In [12]:
train.reset_index(drop=True,inplace=True)
val.reset_index(drop=True,inplace=True)
test.reset_index(drop=True,inplace=True)

In [13]:
train.head(3)

,question1,question2,is_duplicate
0,Can I travel outside USA while my OPT Extensio...,Is is okay to travel to India while my OPT ext...,1
1,Who is John Abraham?,Is John Abraham Gujarati?,0
2,What are some things to do in Goa?,What places should one visit in Goa?,1


In [14]:
from sentence_transformers import InputExample
from tqdm import tqdm, trange

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [15]:
train_examples = [InputExample(texts=[str(row['question1']), str(row['question2'])], label=float(row['is_duplicate'])) for _, row in train.iterrows()]
val_examples = [InputExample(texts=[str(row['question1']), str(row['question2'])], label=float(row['is_duplicate'])) for _, row in val.iterrows()]
test_examples = [InputExample(texts=[str(row['question1']), str(row['question2'])], label=float(row['is_duplicate'])) for _, row in test.iterrows()]

In [16]:
from sentence_transformers import SentenceTransformer, losses, evaluation

# Initialize the model
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.05k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
from torch.utils.data import DataLoader

# Create DataLoaders
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=64)
val_dataloader = DataLoader(val_examples, shuffle=False, batch_size=64)
test_dataloader = DataLoader(test_examples, shuffle=False, batch_size=64)

In [18]:
train_loss = losses.CosineSimilarityLoss(model=model)

In [19]:
# Train the model
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=3,
    warmup_steps=100,
    show_progress_bar=True
)


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,0.163900
1000,0.138700
1500,0.131300
2000,0.128000
2500,0.123600
3000,0.119500
3500,0.117200
4000,0.116100
4500,0.113900
5000,0.114100


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

wandb: ERROR Error while calling W&B API: context deadline exceeded (<Response [500]>)


In [24]:
from sentence_transformers import util
from sklearn.metrics import f1_score

def evaluate_model(model, examples, threshold=0.5):
    cosine_scores = []
    labels = []

    for example in examples:
        # Extract texts and labels
        texts = example.texts
        label = example.label

        # Encode both texts and compute cosine similarity without showing progress bar
        embeddings1 = model.encode(texts[0], convert_to_tensor=True, show_progress_bar=False)
        embeddings2 = model.encode(texts[1], convert_to_tensor=True, show_progress_bar=False)
        score = util.cos_sim(embeddings1, embeddings2)

        # Collect scores and labels
        cosine_scores.append(score.item())
        labels.append(label)

    # Apply threshold to determine predicted labels
    preds = [1 if score > threshold else 0 for score in cosine_scores]

    # Calculate F1 score
    f1 = f1_score(labels, preds)
    return f1

# # Evaluate on validation and test sets
# val_f1 = evaluate_model(model, val_examples)
# test_f1 = evaluate_model(model, test_examples)

# print(f"Validation F1 Score: {val_f1:.4f}")
# print(f"Test F1 Score: {test_f1:.4f}")


In [23]:
# Initialize the model
model2 = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

# Define the Contrastive Loss
train_loss = losses.ContrastiveLoss(model=model2)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [25]:
model2.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=3,
    warmup_steps=100,
    show_progress_bar=True 
)


Exception ignored in: <function tqdm.__del__ at 0x7a4fa7d87910>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/tqdm/std.py", line 1148, in __del__
    self.close()
  File "/opt/conda/lib/python3.10/site-packages/tqdm/notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'


Step,Training Loss
500,0.017000
1000,0.014900
1500,0.014000
2000,0.013600
2500,0.013300
3000,0.012600
3500,0.012700
4000,0.012500
4500,0.012200
5000,0.012000


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [26]:
val_f1 = evaluate_model(model2, val_examples)
test_f1 = evaluate_model(model2, test_examples)

print(f"Validation F1 Score: {val_f1:.4f}")
print(f"Test F1 Score: {test_f1:.4f}")

Validation F1 Score: 0.7518
Test F1 Score: 0.7465


In [17]:
from sentence_transformers import SentenceTransformer, losses

# Initialize the model
model3 = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

# Define the MultipleNegativesRankingLoss
train_loss = losses.MultipleNegativesRankingLoss(model=model3)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.05k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
# Train the model
model3.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=3,
    warmup_steps=100,
    show_progress_bar=True
)


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,0.611400
1000,0.543900
1500,0.543700
2000,0.522500
2500,0.508400
3000,0.492700
3500,0.478500
4000,0.479300
4500,0.481800
5000,0.482800


In [25]:
val_f1 = evaluate_model(model3, val_examples)
test_f1 = evaluate_model(model3, test_examples)

print(f"Validation F1 Score: {val_f1:.4f}")
print(f"Test F1 Score: {test_f1:.4f}")

Validation F1 Score: 0.6325
Test F1 Score: 0.6328
